# Imports

In [1]:
import utils, importlib, sys
importlib.reload(sys.modules['utils'])
from utils import *

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\moi\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
2021-09-03 14:29:57 INFO: Downloading default packages for language: en (English)...
2021-09-03 14:29:58 INFO: File exists: C:\Users\moi\stanza_resources\en\default.zip.
2021-09-03 14:30:02 INFO: Finished downloading models and saved to C:\Users\moi\stanza_resources.
[nltk_data] Downloading package verbnet to
[nltk_data]     C:\Users\moi\AppData\Roaming\nltk_data...
[nltk_data]   Package verbnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\moi\AppData\Roaming\nltk_data...
[nl

In [3]:
models={
    'dt' : DecisionTreeClassifier(random_state=0),
    'mlp': MLPClassifier(max_iter=100,random_state=0),
    'knn': KNeighborsClassifier(),
    'lr' : LogisticRegression(penalty='l2', random_state=0),
    'gnb': GaussianNB(),
    'rf' : RandomForestClassifier(random_state=0),
    'svm': SVC(random_state=0,probability=True)
    }

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))

In [4]:
nlp1 = stanza.Pipeline('en')

2021-09-03 14:30:13 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-09-03 14:30:13 INFO: Use device: cpu
2021-09-03 14:30:13 INFO: Loading: tokenize
2021-09-03 14:30:13 INFO: Loading: pos
2021-09-03 14:30:13 INFO: Loading: lemma
2021-09-03 14:30:14 INFO: Loading: depparse
2021-09-03 14:30:14 INFO: Loading: sentiment
2021-09-03 14:30:15 INFO: Loading: ner
2021-09-03 14:30:16 INFO: Done loading processors!


# Linguistic features

In [5]:
reduced=pd.read_csv("3C-data.csv",sep=";",index_col=0)

In [6]:
reduced.Context=reduced.Context.progress_apply(lambda x: applynlp(x,nlp))

  0%|          | 0/6666 [00:00<?, ?it/s]

In [11]:
ast.literal_eval(reduced["Candidate Antecedent"][0])

[1, 2]

In [12]:
reduced["Pronoun"]=reduced.apply(lambda x: x["Context"][int(x["Position"])],axis=1)
reduced["Candidate Antecedent"]=reduced.apply(lambda x: x["Context"][ast.literal_eval(x["Candidate Antecedent"])[0]:ast.literal_eval(x["Candidate Antecedent"])[1]],axis=1)

In [13]:
gt3=reduced.copy()

In [ ]:
gt3=extract_LF(gt3,nlp1)

# InterRater agreement

In [ ]:
reduced2=pd.read_csv("GT3_v21.8.23",sep=";",index_col=0)

In [ ]:
reduced2.Annotator1=reduced2.Annotator1.apply(lambda x: int(x[0]))
reduced2.Annotator2=reduced2.Annotator2.apply(lambda x: int(x[0]))

In [ ]:
reduced2.Annotator1.value_counts()

In [ ]:
cohen_kappa_score(reduced2.Annotator1,reduced2.Annotator2)

In [ ]:
cohen_kappa_score(reduced2.Annotator1,reduced2.Annotator2,labels=[1,2,3])

## Ambiguous

In [747]:
cohen_kappa_score(reduced2.Annotator1.replace({2:1}),reduced2.Annotator2.replace({2:1}))

0.13478095750868901

## Yes

In [748]:
cohen_kappa_score(reduced2.Annotator1.replace({2:3}),reduced2.Annotator2.replace({2:3}))

0.5526485852339384

## No

In [749]:
cohen_kappa_score(reduced2.Annotator1.replace({1:3}),reduced2.Annotator2.replace({1:3}))

0.5576755953782466

# Embeddings

In [67]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')

In [68]:
model = transformers.BertModel.from_pretrained('bert-base-cased',output_hidden_states = True,)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [69]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [70]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    return tokenized_text, tokens_tensor, segments_tensors
    
def get_hidden_states(tokens_tensor, segments_tensors, model):    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]
    
    return hidden_states
def get_token_embeddings(hidden_states):
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    return token_embeddings

In [71]:
def get_word_emb(text,tokenizer=tokenizer,model=model):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text,tokenizer)
    hidden_states = get_hidden_states(tokens_tensor, segments_tensors, model)
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
    return list_token_embeddings

In [72]:
def get_word_emb_ml(text,tokenizer=tokenizer,model=model,mode="concate"):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text,tokenizer)
    hidden_states = get_hidden_states(tokens_tensor, segments_tensors, model)
    token_embeddings = get_token_embeddings(hidden_states)
    if mode == "concate":
        return concat4(token_embeddings)
    elif mode == "sum":
        return sum4(token_embeddings)
    else:
        print("invalid mode option")

In [73]:
def get_sent_emb(text, tokenizer=tokenizer, model=model):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(
        text, tokenizer)
    hidden_states = get_hidden_states(tokens_tensor, segments_tensors, model)
    token_vecs = hidden_states[-2][0]
    # Calculate the average of all token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    return sentence_embedding

In [74]:
def concat4(token_embeddings):
    token_vecs_cat = []
    # `token_embeddings` is a [22 x 12 x 768] tensor.
    for token in token_embeddings:
        # `token` is a [12 x 768] tensor
        # Concatenate the vectors (that is, append them together) from the last
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]),
                            dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(cat_vec)
    #     print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
    return token_vecs_cat

In [75]:
def sum4(token_embeddings):
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum

In [76]:
def get_4layers_emb(text, tokenizer=tokenizer, model=model,concat=True):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(
        text, tokenizer)
    hidden_states = get_hidden_states(tokens_tensor, segments_tensors, model)
    token_embeddings = get_token_embeddings(hidden_states)
    token_vecs=np.nan
    if concat:
        token_vecs=concat4(token_embeddings)
    else:
        token_vecs=sum4(token_embeddings)
    # Calculate the average of all token vectors.
    sentence_embedding = torch.mean(torch.stack(token_vecs), dim=0)
    return sentence_embedding

In [77]:
def prepare_data_to_save(dataframe):

    dataframe['Pronoun'] = dataframe.apply(lambda x: x['Pronoun'].i, axis=1)

    dataframe['Candidate Antecedent'] = dataframe.apply(
        lambda x:
        [x['Candidate Antecedent'].start, x['Candidate Antecedent'].end],
        axis=1)

    return dataframe

In [78]:
def loadSavedData(dataframe):

    dataframe['Pronoun']=dataframe.apply(lambda x: x['Context'][x['Pronoun']], axis=1)

    dataframe['Candidate Antecedent']=dataframe.apply(lambda x: x['Context'][x['Candidate Antecedent'][0]:x['Candidate Antecedent'][1]], axis=1)

    return dataframe

In [62]:
gt3_backup=gt3.copy()

In [79]:
gt3.Answer.value_counts()

2    5745
0     747
1     526
Name: Answer, dtype: int64

In [85]:
def hashsent1(c,p):
    return c[:p.i].text+" "+p.text+"#1 "+c[p.i+1:].text
def hashsent2(c,p):
    return c[:p.i].text+" "+p.text+"#2 "+c[p.i+1:].text
def hashdouble(c,p,ca):
    if p.i>ca.start:
        return c[:ca.start].text+" "+ca.text+"#2 "+c[ca.end:p.i].text+" "+p.text+"#1 "+c[p.i+1:].text
    if p.i<ca.start:
        return c[:p.i].text+" "+p.text+"#1 "+c[p.i+1:ca.start].text+" "+ca.text+"#2 "+c[ca.end:].text
    print(p, 'in', ca)

## Second-to-last layer

In [88]:
Hs2v=gt3.apply(
    lambda x: get_sent_emb(hashdouble(x['Context'],x['Pronoun'],x['Candidate Antecedent']).strip() + " [SEP] " + x['Pronoun'].text +
                           "#1 [SEP] " + x['Candidate Antecedent'].text+"#2"),
    axis=1)
Hs2=Hs2v.apply(lambda s: pd.Series(
    {i: float(s[i])
     for i in range(0, len(Hs2v[Hs2v.index[0]]))}))

## Concat Last Four Hidden Layers

In [86]:
Hc4v = gt3.apply(
    lambda x: get_4layers_emb(hashdouble(x['Context'],x['Pronoun'],x['Candidate Antecedent']).strip() + " [SEP] " + x['Pronoun'].text +
                           "#1 [SEP] " + x['Candidate Antecedent'].text+"#2"),
    axis=1)
Hc4=Hc4v.apply(lambda s: pd.Series(
    {i: float(s[i])
     for i in range(0, len(Hc4v[Hc4v.index[0]]))}))

## Weighted Sum Last Four Hidden Layers

In [87]:
Hs4v = gt3.apply(
    lambda x: get_4layers_emb(hashdouble(x['Context'],x['Pronoun'],x['Candidate Antecedent']).strip() + " [SEP] " + x['Pronoun'].text +
                           "#1 [SEP] " + x['Candidate Antecedent'].text+"#2",concat=False),
    axis=1)
Hs4=Hs4v.apply(lambda s: pd.Series(
    {i: float(s[i])
     for i in range(0, len(Hs4v[Hs4v.index[0]]))}))

In [80]:
y=gt3.Answer

In [81]:
y=y.apply(lambda x: int(x))

## SBert

In [166]:
sb=pd.read_csv("Sbert.csv",sep=";",index_col=0)

In [168]:
e2sb=pd.read_csv("E2_Sbert",sep=";",index_col=0)

In [169]:
e2sb

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.098836,-0.034070,-0.097639,0.061659,0.102550,0.032165,-0.084558,0.151422,0.104008,0.076060,...,-0.089872,0.079016,0.112327,0.133312,-0.023776,-0.044428,0.092579,-0.043635,-0.129403,-0.091527
1,0.113104,-0.071522,-0.050519,0.003001,0.057471,0.027816,-0.020002,0.108500,0.120959,0.101816,...,-0.066357,0.089366,0.057860,0.135067,-0.036175,-0.046824,0.089482,-0.001744,-0.109935,-0.065041
2,0.103032,-0.052044,-0.089228,0.047880,0.087766,0.042110,-0.050626,0.133466,0.119851,0.069993,...,-0.071466,0.069520,0.092058,0.141068,-0.016481,-0.020127,0.109146,-0.034173,-0.120903,-0.067266
3,0.175229,0.130393,-0.076083,-0.025140,-0.024879,0.156555,0.152766,0.095512,-0.042621,-0.153471,...,0.010930,0.014729,0.065912,-0.074227,-0.065778,-0.164393,0.059183,-0.014788,-0.042896,-0.033001
4,0.157831,0.053361,-0.057349,-0.029612,-0.006451,0.138998,0.156122,0.107582,-0.098198,-0.185145,...,-0.029284,0.019010,0.074145,-0.118780,-0.076750,-0.141577,0.065602,-0.003140,-0.073971,-0.003994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,0.031356,-0.180729,-0.083762,-0.139116,0.001115,0.002384,0.011571,-0.041821,-0.124344,-0.045927,...,-0.049907,0.169860,0.023706,0.133420,-0.060225,-0.042851,0.105605,-0.042384,0.001513,-0.043737
462,-0.018403,-0.198877,-0.088098,-0.129980,0.033815,-0.019654,-0.012218,-0.039931,-0.094763,-0.008892,...,-0.015595,0.166933,0.014916,0.141364,-0.086510,-0.088442,0.113176,-0.017413,0.000763,-0.056675
463,0.022829,-0.189229,-0.069376,-0.154263,0.004649,0.008077,-0.008061,-0.036032,-0.171544,0.007881,...,-0.010617,0.142205,0.041048,0.099297,-0.063883,-0.007534,0.084413,-0.050950,-0.014620,-0.092045
464,0.004589,-0.152306,-0.096401,-0.140081,0.007004,0.021807,0.033049,-0.031986,-0.122380,-0.029709,...,-0.054144,0.175820,0.033444,0.143270,-0.048012,-0.078000,0.098245,-0.060529,-0.024583,-0.057923


# Train test split

In [131]:
X=gt3.drop(['Answer','Context','Pronoun','Candidate Antecedent','Answer'],axis=1)

In [132]:
y=gt3.Answer

In [133]:
X.isNextVerbAnimate=X.isNextVerbAnimate.astype(bool)

In [135]:
E2=loadObj("E2_S.obj")

In [136]:
len(E2)

466

In [137]:
X_2=E2.drop(["Id","Context","Pronoun","Candidate Antecedent","Answer","GT"],axis=1)

In [ ]:
X_2=X_2.loc[E2[~E2.Id.isin(["railway#41","railway#42","railway#43-b","railway#44","railway#45","railway#46-b","railway#47"])].index]

In [138]:
X_2.isNextVerbAnimate=X_2.isNextVerbAnimate.astype(bool)

In [139]:
y_2=E2.Answer

In [140]:
object_cols = []
to_remove=[]
for col, types in zip(
        X_2.dtypes.index,
        X_2.dtypes):
    if types == object:
        if len(X_2[col].unique())<30:
            object_cols.append(col)
        else:
            to_remove.append(col)

In [141]:
X_2=X_2.drop(to_remove,axis=1)

In [142]:
X_2=pd.get_dummies(X_2,columns=object_cols)

In [143]:
for i,j in zip(X_2.isna().sum().index,X_2.isna().sum()):
    if j>0:
        print(i,j)

numWordsB4Comp 435
numWordsB4Inf 320
numWordsB4Prep 207
numWordsB4Ing 388


In [144]:
X_2=X_2.fillna(value=0)


In [155]:
for c2 in list(X_2.columns):
    if c2 not in list(X.columns):
        X_2.drop(c2,axis=1,inplace=True)

In [156]:
for c1 in X.columns:
    if c1 not in X_2.columns:
        print(c1)
        X_2[c1]=0

Id
position_first
position_last
nextPOS_DET
nextPOS_NUM
nextPOS_PRON
nextPOS_SCONJ
headPOS_ADJ
headPOS_PRON
headPOS_PROPN
headPOS_SCONJ
precedingPOS_ADJ
precedingPOS_AUX
precedingPOS_PART
precedingPOS_PRON
precedingPOS_PROPN
precedingPOS_X


In [157]:
X_2.Id=E2.Id

In [145]:
object_cols = []
to_remove=[]
for col, types in zip(
        X.dtypes.index,
        X.dtypes):
    if types == object:
        if len(X[col].unique())<30:
            object_cols.append(col)
        else:
            to_remove.append(col)

In [146]:
to_remove.pop(0)

'Id'

In [147]:
X=X.drop(to_remove,axis=1)

In [214]:
saveObj(X.columns,"trainingCols.list")

In [149]:
X=pd.get_dummies(X,columns=object_cols)

In [150]:
for i,j in zip(X.isna().sum().index,X.isna().sum()):
    if j>0:
        print(i,j)

numWordsB4Comp 6209
numWordsB4Inf 4831
numWordsB4Prep 3060
numWordsB4Ing 5846


In [151]:
X=X.fillna(value=0)


In [158]:
Train_Ids=loadObj("T_Ids.obj")

In [159]:
X_train,X_test=X[X.Id.isin(Train_Ids)],X[~X.Id.isin(Train_Ids)]
y_test = y.iloc[X_test.index].astype('int')
y_train = y.iloc[X_train.index].astype('int')

# Balance

## OverSampling

In [1234]:
X_SOS,y_SOS=SMOTE(sampling_strategy='auto', random_state=0).fit_resample(X_train.drop("Id",axis=1),y_train)

## Undersampling

In [87]:
X_RUS,y_RUS=RandomUnderSampler(sampling_strategy='auto', random_state=0).fit_resample(X_train,y_train)

# Training - evaluation

# Baseline

## E1

### Context

In [477]:
yT_pred=[]
for i in X_test.index:
    Id=X_test.Id[i]
    if len(X_test[X_test.Id==Id])!=1:
        yT_pred.append(0)
    else:
        yT_pred.append(random.choice([1,2]))

In [484]:
p=classification_report(y_test,yT_pred,output_dict=True)['0']['precision']
r=classification_report(y_test,yT_pred,output_dict=True)['0']['recall']
f2=fbeta_score(y_test,yT_pred,beta=2,average=None)[0]
print("precision",p)
print("recall",r)
print("f2",f2)

precision 0.5743889479277364
recall 1.0
f2 0.8709313567515307


### One sentence

In [485]:
yT_pred=[]
for i in X_test.index:
    Id=X_test.Id[i]
    c=gt3.Context[i]
    sents=sent_tokenize(c.text)
    CA=gt3["Candidate Antecedent"][i]
    if ' '.join(sents[1:]).count(CA.text)!=1:
        yT_pred.append(0)
    else:
        yT_pred.append(random.choice([1,2]))

In [486]:
p=classification_report(y_test,yT_pred,output_dict=True)['0']['precision']
r=classification_report(y_test,yT_pred,output_dict=True)['0']['recall']
f2=fbeta_score(y_test,yT_pred,beta=2,average=None)[0]
print("precision",p)
print("recall",r)
print("f2",f2)

precision 0.5124223602484472
recall 0.45790934320074006
f2 0.46786389413988655


## E2

In [498]:
gt3c=pd.read_csv("ReqEval-GT_V21.7.19.csv",sep=";")

In [513]:
gt3c=gt3c[gt3c.sent_Id.isin(E2.Id.unique())]

### Context

In [521]:
yT_pred=[]
for i in gt3c.index:
    Id=gt3c.sent_Id[i]
    if len(gt3c[gt3c.sent_Id==Id])!=1:
        yT_pred.append(0)
    else:
        yT_pred.append(random.choice([1,2]))

In [515]:
gt3c.Answer=gt3c.Answer.replace({'Maybe': 0, 'No': 2, 'Yes': 1})

In [522]:
p=classification_report(gt3c.Answer,yT_pred,output_dict=True)['0']['precision']
r=classification_report(gt3c.Answer,yT_pred,output_dict=True)['0']['recall']
f2=fbeta_score(gt3c.Answer,yT_pred,beta=2,average=None)[0]
print("precision",p)
print("recall",r)
print("f2",f2)

precision 0.6148796498905909
recall 0.9894366197183099
f2 0.8819836785938481


In [523]:
classification_report(gt3c.Answer,yT_pred,output_dict=True)['0']

{'precision': 0.6148796498905909,
 'recall': 0.9894366197183099,
 'f1-score': 0.7584345479082321,
 'support': 284}

### One sentence

In [517]:
gt3c.head()

,sent_Id,context,pronoun,CA,Answer,GT
0,library#01,All material that is stored in the repository ...,it,All material,0,AMBIGUOUS
1,library#01,All material that is stored in the repository ...,it,the repository,0,AMBIGUOUS
2,library#01,All material that is stored in the repository ...,it,material,0,AMBIGUOUS
3,library#02,The library may want to accept important digit...,them,The library,2,digital materials
4,library#02,The library may want to accept important digit...,them,important digital materials,1,digital materials


In [518]:
yT_pred=[]
for i in gt3c.index:
    c=gt3c.context[i]
    sents=sent_tokenize(c)
    CA=gt3c.CA[i]
    if ' '.join(sents[1:]).count(CA)!=1:
        yT_pred.append(0)
    else:
        yT_pred.append(random.choice([1,2]))

In [519]:
p=classification_report(gt3c.Answer,yT_pred,output_dict=True)['0']['precision']
r=classification_report(gt3c.Answer,yT_pred,output_dict=True)['0']['recall']
f2=fbeta_score(gt3c.Answer,yT_pred,beta=2,average=None)[0]
print("precision",p)
print("recall",r)
print("f2",f2)

precision 0.6094420600858369
recall 1.0
f2 0.8863920099875156


# Model selection

## TrainSet 

### LF

In [1408]:
avg=0
skf = StratifiedKFold(n_splits=10, shuffle=True)
Xids=X_train.Id.unique()
for n in models:
    m=models[n]
    cv_score = [0.,0.,0.]
    print(n, "==========")
    for train_idx, test_idx in skf.split(X_train.Id.unique(),Yids(X_train,y_train)):
        clf=clone(m)
        train_idx, test_idx=X_train[X_train.Id.isin(Xids[train_idx])].index, X_train[X_train.Id.isin(Xids[test_idx])].index
        xfold_train, yfold_train = X_train.drop("Id",axis=1).loc[train_idx],y_train.loc[train_idx]
        clf.fit(xfold_train, yfold_train)
        y_pred=clf.predict_proba(X_train.drop("Id",axis=1).loc[test_idx])
        test_scores=getpredictionScores(y_train.loc[test_idx].index,y_pred,X_train.loc[test_idx].Id,0.5,0.5,Class='0',B=2)
        cv_score[0] += test_scores[0]
        cv_score[1] += test_scores[1]
        cv_score[2] += test_scores[2]
    print([float("{:.3f}".format(s/10)) for s in cv_score])
    avg+=scores[2]#[2]
print("avg==",avg/len(models))

dt ==========
[0.509, 0.94, 0.803]
mlp ==========
[0.502, 0.962, 0.812]
knn ==========
[0.503, 0.91, 0.783]
lr ==========
[0.495, 0.89, 0.766]
gnb ==========
[0.504, 0.997, 0.833]
rf ==========
[0.499, 0.943, 0.8]
svm ==========
[0.5, 0.944, 0.801]
avg== 0.793


### EF

#### Sbert

In [1416]:
sbtrain=sb.loc[X_train.index]
sbtrain["Id"]=X_train["Id"]

In [1468]:
avg=0
skf = StratifiedKFold(n_splits=10, shuffle=True)
Xids=sbtrain.Id.unique()
for n in models:
    m=models[n]
    cv_score = [0.,0.,0.]
    print(n, "==========")
    for train_idx, test_idx in skf.split(sbtrain.Id.unique(),Yids(sbtrain,y_train)):
        clf=clone(m)
        train_idx, test_idx=sbtrain[sbtrain.Id.isin(Xids[train_idx])].index, sbtrain[sbtrain.Id.isin(Xids[test_idx])].index
        xfold_train, yfold_train = sbtrain.drop("Id",axis=1).loc[train_idx],y_train.loc[train_idx]
        clf.fit(xfold_train, yfold_train)
        y_pred=clf.predict_proba(sbtrain.drop("Id",axis=1).loc[test_idx])
        test_scores=getpredictionScores(y_train.loc[test_idx].index,y_pred,sbtrain.loc[test_idx].Id,0.5,0.5,Class='0',B=2)
        cv_score[0] += test_scores[0]
        cv_score[1] += test_scores[1]
        cv_score[2] += test_scores[2]
    print([float("{:.3f}".format(s/10)) for s in cv_score])
    avg+=scores[2]#[2]
print("avg==",avg/len(models))

dt ==========
[0.51, 0.863, 0.757]
mlp ==========
[0.516, 0.992, 0.836]
knn ==========
[0.502, 0.982, 0.824]
lr ==========
[0.503, 0.953, 0.806]
gnb ==========
[0.508, 0.983, 0.828]
rf ==========
[0.503, 0.945, 0.803]
svm ==========
[0.5, 0.972, 0.817]
avg== 0.673


#### Concat

In [1427]:
Hc4Train=Hc4.copy()
Hc4Train["Id"]=X_train["Id"]

In [1466]:
avg=0
skf = StratifiedKFold(n_splits=10, shuffle=True)
Xids=Hc4Train.Id.unique()
for n in models:
    m=models[n]
    cv_score = [0.,0.,0.]
    print(n, "==========")
    for train_idx, test_idx in skf.split(Hc4Train.Id.unique(),Yids(Hc4Train,y_train)):
        clf=clone(m)
        train_idx, test_idx=Hc4Train[Hc4Train.Id.isin(Xids[train_idx])].index, Hc4Train[Hc4Train.Id.isin(Xids[test_idx])].index
        xfold_train, yfold_train = Hc4Train.drop("Id",axis=1).loc[train_idx],y_train.loc[train_idx]
        clf.fit(xfold_train, yfold_train)
        y_pred=clf.predict_proba(Hc4Train.drop("Id",axis=1).loc[test_idx])
        test_scores=getpredictionScores(y_train.loc[test_idx].index,y_pred,Hc4Train.loc[test_idx].Id,0.5,0.5,Class='0',B=2)
        cv_score[0] += test_scores[0]
        cv_score[1] += test_scores[1]
        cv_score[2] += test_scores[2]
    print([float("{:.3f}".format(s/10)) for s in cv_score])
    avg+=scores[2]#[2]
print("avg==",avg/len(models))

dt ==========
[0.519, 0.89, 0.777]
mlp ==========
[0.499, 0.94, 0.798]
knn ==========
[0.502, 0.977, 0.821]
lr ==========
[0.512, 0.972, 0.823]
gnb ==========
[0.506, 0.987, 0.829]
rf ==========
[0.506, 0.955, 0.809]
svm ==========
[0.504, 0.977, 0.822]
avg== 0.673


#### Sum

In [1430]:
Hs4Train=Hs4.copy()
Hs4Train["Id"]=X_train["Id"]

In [1465]:
avg=0
skf = StratifiedKFold(n_splits=10, shuffle=True)
Xids=Hs4Train.Id.unique()
for n in models:
    m=models[n]
    cv_score = [0.,0.,0.]
    print(n, "==========")
    for train_idx, test_idx in skf.split(Hs4Train.Id.unique(),Yids(Hs4Train,y_train)):
        clf=clone(m)
        train_idx, test_idx=Hs4Train[Hs4Train.Id.isin(Xids[train_idx])].index, Hs4Train[Hs4Train.Id.isin(Xids[test_idx])].index
        xfold_train, yfold_train = Hs4Train.drop("Id",axis=1).loc[train_idx],y_train.loc[train_idx]
        clf.fit(xfold_train, yfold_train)
        y_pred=clf.predict_proba(Hs4Train.drop("Id",axis=1).loc[test_idx])
        test_scores=getpredictionScores(y_train.loc[test_idx].index,y_pred,Hs4Train.loc[test_idx].Id,0.5,0.5,Class='0',B=2)
        cv_score[0] += test_scores[0]
        cv_score[1] += test_scores[1]
        cv_score[2] += test_scores[2]
    print([float("{:.3f}".format(s/10)) for s in cv_score])
    avg+=scores[2]#[2]
print("avg==",avg/len(models))

dt ==========
[0.569, 0.873, 0.775]
mlp ==========
[0.555, 0.969, 0.836]
knn ==========
[0.557, 0.983, 0.847]
lr ==========
[0.525, 0.904, 0.783]
gnb ==========
[0.55, 0.953, 0.828]
rf ==========
[0.577, 1.0, 0.859]
svm ==========
[0.56, 0.926, 0.803]
avg== 0.673


#### s2l

In [1432]:
Hs2Train=Hs2.copy()
Hs2Train["Id"]=X_train["Id"]

In [1467]:
avg=0
skf = StratifiedKFold(n_splits=10, shuffle=True)
Xids=Hs2Train.Id.unique()
for n in models:
    m=models[n]
    cv_score = [0.,0.,0.]
    print(n, "==========")
    for train_idx, test_idx in skf.split(Hs2Train.Id.unique(),Yids(Hs2Train,y_train)):
        clf=clone(m)
        train_idx, test_idx=Hs2Train[Hs2Train.Id.isin(Xids[train_idx])].index, Hs2Train[Hs2Train.Id.isin(Xids[test_idx])].index
        xfold_train, yfold_train = Hs2Train.drop("Id",axis=1).loc[train_idx],y_train.loc[train_idx]
        clf.fit(xfold_train, yfold_train)
        y_pred=clf.predict_proba(Hs2Train.drop("Id",axis=1).loc[test_idx])
        test_scores=getpredictionScores(y_train.loc[test_idx].index,y_pred,Hs2Train.loc[test_idx].Id,0.5,0.5,Class='0',B=2)
        cv_score[0] += test_scores[0]
        cv_score[1] += test_scores[1]
        cv_score[2] += test_scores[2]
    print([float("{:.3f}".format(s/10)) for s in cv_score])
    avg+=scores[2]#[2]
print("avg==",avg/len(models))

dt ==========
[0.498, 0.89, 0.767]
mlp ==========
[0.511, 0.98, 0.827]
knn ==========
[0.498, 0.968, 0.814]
lr ==========
[0.504, 0.959, 0.812]
gnb ==========
[0.501, 0.969, 0.815]
rf ==========
[0.511, 0.967, 0.82]
svm ==========
[0.504, 0.977, 0.822]
avg== 0.673


# HyperParameter Tuning

In [175]:
lrg={'C':np.logspace(-4, 4, 50), 
     'solver' :['newton-cg', 'lbfgs', 'liblinear'],
     'penalty' : ['l2']}
#Decision Tree
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]
dtg=dict(criterion=criterion, max_depth=max_depth)
# dtg= dict(max_depth = list(range(1,34,4))+[None], 
#           criterion = ['gini', 'entropy'],
#           min_samples_split = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
#           max_leaf_nodes= list(range(2, 100,10)),
#           min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True),
#           max_features = ['sqrt', 'log2',None])
# rfg=dict(n_estimators = [100, 1000,10000],
#           max_depth = list(np.linspace(1, 32, 5, endpoint=True))+[None], 
#           criterion = ['gini', 'entropy'],
#           min_samples_split = np.linspace(0.1, 1.0, 3, endpoint=True),
#           #max_leaf_nodes= list(range(2, 100,10)),
#           min_samples_leaf = np.linspace(0.1, 0.5, 3, endpoint=True),
#           max_features = ['sqrt', 'log2']+[None])
#Random Forest
n_estimators = [10, 100, 1000,10000]
max_features = ['sqrt', 'log2']
rfg = dict(n_estimators=n_estimators,max_features=max_features)
etg={'n_estimators': [100, 1000,10000],
    'max_features': ['sqrt', 'log2',None],
    'min_samples_leaf': list(range(20,50,10))+[0.1,None],
    'min_samples_split': list(range(15,36,5))+[None]}
abg={'n_estimators': [100,200,300,500],
    'learning_rate':[0.001,0.01,0.1,0.2,0.5]}
svmg=dict(kernel = ['poly', 'rbf', 'sigmoid','linear'],
    C = [50, 10, 1.0, 0.1, 0.01,0.001],
    gamma = [0.001,0.01,0.1,1,'scale'])
mlpg={
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive']}
knng=dict(n_neighbors = range(1, 21, 2), weights = ['uniform', 'distance'], 
          metric = ['euclidean', 'manhattan', 'minkowski'])
abg={'n_estimators': [100,200,300,500],
    'learning_rate':[0.001,0.01,0.1,0.2,0.5]}
xgbg={  'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.02, 0.05]}
gnbg={'var_smoothing': np.logspace(0,-9, num=100)}

In [176]:
grids={ 'lr':lrg,
        'dt':dtg,
        'rf':rfg,
        'et':etg,
        'ab':abg,
        'svm':svmg,
        'mlp':mlpg,
        'knn':knng,
        #'xgb':xgbg,
        'gnb':gnbg,
        'bnb':None}

## LF

In [198]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [199]:
m=models['gnb']
cv=StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=m, 
              param_grid=grids['gnb'], n_jobs=-1, cv=cv,  
              scoring=ftwo_scorer)
print("gnb  ==========")
grid_search.fit(X_train.drop("Id",axis=1),y_train)
bp=grid_search.best_params_
print(bp)
y_pred=grid_search.best_estimator_.predict_proba(X_test.drop("Id",axis=1))
test_scores=getpredictionScores(y_test.index,y_pred,X_test.Id,0.5,0.5,Class='0',B=2)
print(test_scores)

gnb  ==========
{'var_smoothing': 1.0}
(0.561, 1.0, 0.864)


### E2

In [792]:
X_2.Id=E2.Id

In [795]:
y_2=y_2[X_2.index]

In [796]:
gn=GaussianNB().set_params(**{'var_smoothing': 1.0})
gn.fit(X_train.drop("Id",axis=1),y_train)
y_pred=gn.predict_proba(X_2.drop("Id",axis=1))
test_scores=getpredictionScores(y_2.index,y_pred,X_2.Id,0.5,0.5,E2,Class='0',B=2)
print(test_scores)

(0.618, 0.989, 0.883)


In [797]:
gn=GaussianNB().set_params(**{'var_smoothing': 1.0})
gn.fit(X_train.drop("Id",axis=1),y_train)
y_pred=gn.predict_proba(X_2.drop("Id",axis=1))
test_scores=getpredictionScores(y_2.index,y_pred,X_2.Id,0.5,0.5,E2,Class='0',B=2)
print(test_scores)

(0.618, 0.989, 0.883)


In [205]:
saveObj(gn,"ML_FL-detection.Anaphora")

## EF 

In [603]:
Hs4.shape

(6666, 768)

In [207]:
Hs4C=Hs4.copy()
Hs4C["Id"]=X["Id"]
Hs4_Train=Hs4C[Hs4C.Id.isin(X_train.Id.unique())]
Hs4_Test=Hs4C[Hs4C.Id.isin(X_test.Id.unique())]

In [1464]:
m=models['rf']
cv=StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=m, 
              param_grid=grids['rf'], n_jobs=-1, cv=cv,  
              scoring=ftwo_scorer)
print("rf ==========")
grid_search.fit(Hs4_Train.drop("Id",axis=1),y_train)
bp=grid_search.best_params_
print(bp)
y_pred=grid_search.best_estimator_.predict_proba(Hs4_Test.drop("Id",axis=1))
test_scores=getpredictionScores(y_test.index,y_pred,Hs4_Test.Id,0.5,0.5,Class='0',B=2)
print(test_scores)

rf ==========
{'max_features': 'sqrt', 'n_estimators': 10}
(0.604, 0.917, 0.831)


In [1675]:
m=RandomForestClassifier(random_state=3194).set_params(**{'max_features': 'sqrt', 'n_estimators': 10})
m.fit(Hs4_Train.drop("Id",axis=1),y_train)
y_pred=m.predict_proba(Hs4_Test.drop("Id",axis=1))
test_scores=getpredictionScores(y_test.index,y_pred,Hs4_Test.Id,0.5,0.5,gt4,Class='0',B=2)
print(test_scores)

(0.576, 1.0, 0.872)


### E2

In [1477]:
E2=loadObj("E2_S.obj")

In [1479]:
E2.Pronoun=E2.apply(lambda x: x['Context'][x['Pronoun']],axis=1)

In [1480]:
E2['Candidate Antecedent']=E2.apply(lambda x: x['Context'][x['Candidate Antecedent'][0]:x['Candidate Antecedent'][1]],axis=1)

In [800]:
E2s4v = E2.apply(
    lambda x: get_4layers_emb(hashdouble(x['Context'],x['Pronoun'],x['Candidate Antecedent']).strip() + " [SEP] " + x['Pronoun'].text +
                           "#1 [SEP] " + x['Candidate Antecedent'].text+"#2",concat=False),
    axis=1)
E2s4=E2s4v.apply(lambda s: pd.Series(
    {i: float(s[i])
     for i in range(0, len(E2s4v[E2s4v.index[0]]))}))

In [801]:
E2s4["Id"]=E2["Id"]

In [803]:
E2s4=E2s4.loc[X_2.index]

In [807]:
m=RandomForestClassifier(random_state=3178).set_params(**{'max_features': 'sqrt', 'n_estimators': 10})
m.fit(Hs4_Train.drop("Id",axis=1),y_train)
y_pred=m.predict_proba(E2s4.drop("Id",axis=1))
test_scores=getpredictionScores(y_2.index,y_pred,E2s4.Id,0.5,0.5,E2,Class='0',B=2)
print(test_scores)

(0.622, 0.982, 0.88)


## Ensembling

In [1552]:
def biEnScoringFB(ypred, x, y,B=2):
    tp,fp,tn,fn=0,0,0,0
    ypredx=y.copy()
    for p,i in zip(ypred,y.index):
        ypredx[i]=p

    for Id in x.Id.unique():
        idx=x[x.Id==Id].index
        if 0 in list(y[idx]):
            if 0 in list(ypredx[idx]):
                tp+=1
            else:
                fn+=1
        else:
            if 0 in list(ypredx[idx]):
                fp+=1
            else:
                tn+=1

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    FB=((1+B*B)*tp)/((1+B*B)*tp+B*B*fn+fp)
    return [float("{:.3f}".format(precision)), float("{:.3f}".format(recall)), float("{:.3f}".format(FB))]


In [641]:
def ensembleprobaN(ypred1,ypred2,theta=0):
    ypred=[]
    for l1,l2 in zip(ypred1,ypred2):
        m1=np.max(l1)
        m2=np.max(l2)
        d1=np.where(l1==m1)[0][0]
        d2=np.where(l2==m2)[0][0]
        if d1==0 or d2==0: 
            ypred.append([np.max([m1,m2]),0,0])
            continue
        elif theta!=0:
            if d1!=d2 and abs(m1-m2)<theta:
                ypred.append([np.max([m1,m2]),0,0])
                continue
        else:
            e=[0,0,0]
            e[d1]+=m1
            e[d2]+=m2
            ypred.append(e)
    return np.array(ypred)


### E1

In [1581]:
gn=GaussianNB().set_params(**{'var_smoothing': 1.0})
gn.fit(X_train.drop("Id",axis=1),y_train)
y_pred1=gn.predict_proba(X_test.drop("Id",axis=1))

m=RandomForestClassifier(random_state=7).set_params(**{'max_features': 'sqrt', 'n_estimators': 10})
m.fit(Hs4_Train.drop("Id",axis=1),y_train)
y_pred2=m.predict_proba(Hs4_Test.drop("Id",axis=1))


ypred=ensembleprobaN(y_pred1,y_pred2,theta=0.1)
test_scores=getpredictionScores(y_test.index,ypred,X_test.Id,0.5,0.5,gt4,Class='0',B=2)
print(theta, test_scores)

(0.58, 1.0, 0.873)


### E2

In [ ]:
gn=GaussianNB().set_params(**{'var_smoothing': 1.0})
gn.fit(X_train.drop("Id",axis=1),y_train)
y_pred1=gn.predict_proba(X_2.drop("Id",axis=1))

m=RandomForestClassifier(random_state=3194).set_params(**{'max_features': 'sqrt', 'n_estimators': 10})
m.fit(Hs4_Train.drop("Id",axis=1),y_train)
y_pred2=m.predict_proba(E2s4.drop("Id",axis=1))


ypred=ensembleprobaN(y_pred1,y_pred2,theta=0.1)
test_scores=getpredictionScores(y_2.index,ypred,E2s4.Id,0.5,0.5,E2,Class='0',B=2)
print(theta,test_scores)

# Resolution

In [160]:
def proba2pred(proba):
    pred=proba.copy()
    for p,i in zip(proba,proba.index):
        pred[i]=np.argmax(p)
    return pred

In [161]:
def mindex(y_pred,y_test):
    return pd.Series(y_pred.tolist(), index =y_test.index)

In [163]:
li=[]
for Id in X_train.Id.unique():
    if 0 not in y_train[X_train[X_train.Id==Id].index].unique():
           li.append(Id)

In [164]:
X_train[X_train.Id.isin(li)]

,Id,Position,Distance,sameGender,sameNumber,isPerson,isDefinite,isPrepositional,isSubject,isDirectObject,...,precedingPOS_CCONJ,precedingPOS_DET,precedingPOS_NOUN,precedingPOS_PART,precedingPOS_PRON,precedingPOS_PROPN,precedingPOS_PUNCT,precedingPOS_SCONJ,precedingPOS_VERB,precedingPOS_X
0,2007-ertms#0,10.0,8,False,False,False,False,False,True,False,...,0,0,0,0,0,0,0,0,1,0
1,2007-ertms#0,10.0,4,False,True,False,True,False,False,True,...,0,0,0,0,0,0,0,0,1,0
2,2007-ertms#0,10.0,2,False,True,False,False,False,False,False,...,0,0,0,0,0,0,0,0,1,0
26,2007-ertms#5,58.0,13,True,False,False,False,False,True,False,...,0,1,0,0,0,0,0,0,0,0
27,2007-ertms#5,58.0,11,True,True,False,False,False,False,True,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6660,EDRS_MOCReqs#711,29.0,16,True,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
6661,EDRS_MOCReqs#711,29.0,19,True,False,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0
6662,EDRS_MOCReqs#711,29.0,13,True,False,False,True,False,True,False,...,0,0,0,0,0,0,0,0,0,0
6663,EDRS_MOCReqs#711,29.0,4,True,True,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [165]:
def resolve(y_proba,X_test,y_test,gt3=gt3,theta=0.5):
    ci=[0,0]
    for Id in X_test.Id.unique():
        if 0 not in y_test[X_test[X_test.Id==Id].index].unique():
            preds=proba2pred(y_proba[X_test[X_test.Id==Id].index])
            if 1 in preds.values:
                if list(preds.values).count(1)==1:
                    if y_test[preds[preds==1].index[0]]==1:
                        ci[0]+=1
                    else:
                        ci[1]+=1
                else:
                    if theta!=0:
                        above_theta=[]
                        for i in preds[preds==1].index:
                            #print(y_proba[i][1],preds[i],i)
                            if y_proba[i][1]>=theta:
                                above_theta.append(i)
                        if len(above_theta)!=1:
                            ci[1]+=1
                        elif preds[above_theta[0]]==1:
                            ci[0]+=1
                        else:
                            ci[1]+=1
                    else:
                        maxs=[]
                        mx=y_proba[preds[preds==1].index].apply(lambda x: x[1]).max()
                        for i,p in zip(preds[preds==1].index,y_proba[preds[preds==1].index]):
                            if p[1]==mx:
                                maxs.append(i)
                        if len(maxs)>1:
                            trm=[]
                            for i in range(0,len(maxs)-1):
                                for j in range(i+1,len(maxs)):
                                    if gt3["Candidate Antecedent"][maxs[i]]==gt3["Candidate Antecedent"][maxs[j]]:
                                        trm.append(i)
                            maxs=np.delete(maxs,trm)
                            if len(maxs)>1:
                                ci[1]+=1
                            elif preds[maxs[0]]==1:
                                ci[0]+=1
                            else:
                                ci[1]+=1
                        elif preds[maxs[0]]==1:
                            ci[0]+=1
                        else:
                            ci[1]+=1
            else:
                ci[1]+=1
    return ci[0]/(ci[0]+ci[1])

In [292]:
m=RandomForestClassifier().set_params(**{'max_features': 'sqrt', 'n_estimators': 10})
m.fit(X_train.drop("Id",axis=1),y_train)
y_pred=m.predict_proba(X_test.drop("Id",axis=1))
y_pred=mindex(y_pred,y_test)

In [306]:
resolve(y_pred,X_test,y_test,theta=0.5)

0.15625

In [307]:
resolve(y_pred,X_test,y_test,theta=0)

0.19791666666666666

## Model Selection

### LF

In [365]:
models=reset()
for stratname,sampler in [("RUS",RandomUnderSampler(sampling_strategy='not minority', random_state=0))]:
    print(stratname,"+++++++++++++++++++++++++")
    avg=0
    train_idx, test_idx = groupTTsplit(X_train, y_train,X_train["Id"].unique(),ratio=0.2)
    XS_train, XS_test, ys_train, ys_test = X_train.loc[train_idx].drop("Id",axis=1),X_train.loc[test_idx], y_train.loc[train_idx], y_train.loc[test_idx]
    XS_train, ys_train=sampler.fit_resample(XS_train, ys_train)
    for n in models:
        m=models[n]
        print(stratname, n, "==========")
        m.fit(XS_train,ys_train)
        #scores=scoringF1(m, XS_test, ys_test,Class=0,MultiClass=False)
        y_pred=m.predict_proba(XS_test.drop("Id",axis=1))
        y_pred=mindex(y_pred,ys_test)
        print(resolve(y_pred,XS_test,ys_test,theta=0.5),0.5)
        print(resolve(y_pred,XS_test,ys_test,theta=0),0)

RUS +++++++++++++++++++++++++
RUS dt ==========
0.2542372881355932 0.5
0.3220338983050847 0
RUS mlp ==========
0.3559322033898305 0.5
0.8135593220338984 0
RUS knn ==========
0.288135593220339 0.5
0.6101694915254238 0
RUS lr ==========
0.288135593220339 0.5
0.864406779661017 0
RUS gnb ==========
0.05084745762711865 0.5
0.1864406779661017 0
RUS rf ==========
0.3728813559322034 0.5
0.711864406779661 0
RUS svm ==========
0.22033898305084745 0.5
0.9152542372881356 0


### SBERT

In [311]:
sbtrain=sb.loc[X_train.index]
sbtrain["Id"]=X_train["Id"]

In [364]:
models=reset()
for stratname,sampler in [("RUS",RandomUnderSampler(sampling_strategy='not minority', random_state=0))]:
    print(stratname,"+++++++++++++++++++++++++")
    avg=0
    train_idx, test_idx = groupTTsplit(sbtrain, y_train,sbtrain["Id"].unique(),ratio=0.2)
    XS_train, XS_test, ys_train, ys_test = sbtrain.loc[train_idx].drop("Id",axis=1),sbtrain.loc[test_idx], y_train.loc[train_idx], y_train.loc[test_idx]
    XS_train, ys_train=sampler.fit_resample(XS_train, ys_train)
    for n in models:
        m=models[n]
        print(stratname, n, "==========")
        m.fit(XS_train,ys_train)
        #scores=scoringF1(m, XS_test, ys_test,Class=0,MultiClass=False)
        y_pred=m.predict_proba(XS_test.drop("Id",axis=1))
        y_pred=mindex(y_pred,ys_test)
        #print(resolve(y_pred,XS_test,ys_test,theta=0.5),0.5)
        print(resolve(y_pred,XS_test,ys_test,theta=0),0)

RUS +++++++++++++++++++++++++
RUS dt ==========
0.06779661016949153 0
RUS mlp ==========
0.7457627118644068 0
RUS knn ==========
0.13559322033898305 0
RUS lr ==========
0.6610169491525424 0
RUS gnb ==========
0.6271186440677966 0
RUS rf ==========
0.6610169491525424 0
RUS svm ==========
0.6949152542372882 0


### Concat

In [317]:
Hc4Train=Hc4.copy()
Hc4Train=Hc4Train.loc[X_train.index]
Hc4Train["Id"]=X_train["Id"]

In [363]:
models=reset()
for stratname,sampler in [("RUS",RandomUnderSampler(sampling_strategy='not minority', random_state=0))]:
    print(stratname,"+++++++++++++++++++++++++")
    avg=0
    train_idx, test_idx = groupTTsplit(Hc4Train, y_train,Hc4Train["Id"].unique(),ratio=0.2)
    XS_train, XS_test, ys_train, ys_test = Hc4Train.loc[train_idx].drop("Id",axis=1),Hc4Train.loc[test_idx], y_train.loc[train_idx], y_train.loc[test_idx]
    XS_train, ys_train=sampler.fit_resample(XS_train, ys_train)
    for n in models:
        m=models[n]
        print(stratname, n, "==========")
        m.fit(XS_train,ys_train)
        #scores=scoringF1(m, XS_test, ys_test,Class=0,MultiClass=False)
        y_pred=m.predict_proba(XS_test.drop("Id",axis=1))
        y_pred=mindex(y_pred,ys_test)
        #print(resolve(y_pred,XS_test,ys_test,theta=0.5),0.5)
        print(resolve(y_pred,XS_test,ys_test,theta=0),0)

RUS +++++++++++++++++++++++++
RUS dt ==========
0.0847457627118644 0
RUS mlp ==========
0.6101694915254238 0
RUS knn ==========
0.1694915254237288 0
RUS lr ==========
0.6610169491525424 0
RUS gnb ==========
0.1864406779661017 0
RUS rf ==========
0.6779661016949152 0
RUS svm ==========
0.6610169491525424 0


### Sum

In [319]:
Hs4Train=Hs4.copy()
Hs4Train=Hs4Train.loc[X_train.index]
Hs4Train["Id"]=X_train["Id"]

In [362]:
models=reset()
for stratname,sampler in [("RUS",RandomUnderSampler(sampling_strategy='not minority', random_state=0))]:
    print(stratname,"+++++++++++++++++++++++++")
    avg=0
    train_idx, test_idx = groupTTsplit(Hs4Train, y_train,Hs4Train["Id"].unique(),ratio=0.2)
    XS_train, XS_test, ys_train, ys_test = Hs4Train.loc[train_idx].drop("Id",axis=1),Hs4Train.loc[test_idx], y_train.loc[train_idx], y_train.loc[test_idx]
    XS_train, ys_train=sampler.fit_resample(XS_train, ys_train)
    for n in models:
        m=models[n]
        print(stratname, n, "==========")
        m.fit(XS_train,ys_train)
        #scores=scoringF1(m, XS_test, ys_test,Class=0,MultiClass=False)
        y_pred=m.predict_proba(XS_test.drop("Id",axis=1))
        y_pred=mindex(y_pred,ys_test)
        #print(resolve(y_pred,XS_test,ys_test,theta=0.5),0.5)
        print(resolve(y_pred,XS_test,ys_test,theta=0),0)

RUS +++++++++++++++++++++++++
RUS dt ==========
0.06779661016949153 0
RUS mlp ==========
0.576271186440678 0
RUS knn ==========
0.15254237288135594 0
RUS lr ==========
0.6271186440677966 0
RUS gnb ==========
0.5254237288135594 0
RUS rf ==========
0.576271186440678 0
RUS svm ==========
0.6610169491525424 0


### S2L

In [321]:
Hs2Train=Hs2.copy()
Hs2Train=Hs2Train.loc[X_train.index]
Hs2Train["Id"]=X_train["Id"]

In [361]:
models=reset()
for stratname,sampler in [("RUS",RandomUnderSampler(sampling_strategy='not minority', random_state=0))]:
    print(stratname,"+++++++++++++++++++++++++")
    avg=0
    train_idx, test_idx = groupTTsplit(Hs2Train, y_train,Hs2Train["Id"].unique(),ratio=0.2)
    XS_train, XS_test, ys_train, ys_test = Hs2Train.loc[train_idx].drop("Id",axis=1),Hs2Train.loc[test_idx], y_train.loc[train_idx], y_train.loc[test_idx]
    XS_train, ys_train=sampler.fit_resample(XS_train, ys_train)
    for n in models:
        m=models[n]
        print(stratname, n, "==========")
        m.fit(XS_train,ys_train)
        #scores=scoringF1(m, XS_test, ys_test,Class=0,MultiClass=False)
        y_pred=m.predict_proba(XS_test.drop("Id",axis=1))
        y_pred=mindex(y_pred,ys_test)
        #print(resolve(y_pred,XS_test,ys_test,theta=0.5),0.5)
        print(resolve(y_pred,XS_test,ys_test,theta=0),0)

RUS +++++++++++++++++++++++++
RUS dt ==========
0.0 0
RUS mlp ==========
0.5932203389830508 0
RUS knn ==========
0.1694915254237288 0
RUS lr ==========
0.6610169491525424 0
RUS gnb ==========
0.559322033898305 0
RUS rf ==========
0.6779661016949152 0
RUS svm ==========
0.711864406779661 0


## Hyper Parameters Optimization

### LF

In [384]:
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
m=models['svm']
cv=StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=m, 
              param_grid=grids['svm'], n_jobs=-1, cv=cv,  
              scoring=ftwo_scorer)
print("svm  ==========")
grid_search.fit(XS_train.drop("Id",axis=1),ys_train)
bp=grid_search.best_params_
print(bp)
y_pred=grid_search.best_estimator_.predict_proba(X_test.drop("Id",axis=1))
y_pred=mindex(y_pred,y_test)
test_scores=resolve(y_pred,X_test.drop([1159,3530]),y_test.drop([1159,3530]),theta=0)
print(test_scores)

svm  ==========
{'C': 50, 'gamma': 0.001, 'kernel': 'poly'}
0.8125


In [388]:
models['svm']

SVC(probability=True, random_state=0)

In [395]:
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
m=models['svm'].set_params(**{'C': 50, 'gamma': 0.001, 'kernel': 'poly'})
m.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred=m.predict_proba(X_test.drop("Id",axis=1))
y_pred=mindex(y_pred,y_test)
test_scores=resolve(y_pred,X_test,y_test,theta=0)
print(test_scores)

0.8125


In [829]:
models=reset()
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
m=models['svm'].set_params(**{'C': 50, 'gamma': 0.001, 'kernel': 'poly'})
m.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred=m.predict_proba(X_2.drop("Id",axis=1))
y_pred=mindex(y_pred,y_2)
test_scores=resolve(y_pred,X_2,y_2,theta=0)
print(test_scores)

0.574468085106383


In [210]:
models=resetmodels()
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
m=models['svm'].set_params(**{'C': 50, 'gamma': 0.001, 'kernel': 'poly'})
m.fit(XS_train.drop("Id",axis=1),ys_train)

SVC(C=50, gamma=0.001, kernel='poly', probability=True, random_state=0)

In [211]:
saveObj(m,"ML_LF-resolution.Anaphora")

### SBERT

In [172]:
sbtrain=sb.loc[X_train.index]
sbtrain["Id"]=X_train["Id"]

In [170]:
sbtest=sb.loc[X_test.index]
sbtest["Id"]=X_test["Id"]

In [177]:
models=reset()
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(sbtrain,y_train)
m=models['mlp']
cv=StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=m, 
              param_grid=grids['mlp'], n_jobs=-1, cv=cv,  
              scoring=ftwo_scorer)
print("mlp  ==========")
grid_search.fit(XS_train.drop("Id",axis=1),ys_train)
bp=grid_search.best_params_
print(bp)

mlp  ==========
{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}


In [178]:
y_pred=grid_search.best_estimator_.predict_proba(sbtest.drop("Id",axis=1))
y_pred=mindex(y_pred,y_test)
test_scores=resolve(y_pred,sbtest,y_test,theta=0)
print(test_scores)

0.5555555555555556


In [180]:
y_pred=grid_search.best_estimator_.predict_proba(sbtrain.drop("Id",axis=1))
y_pred=mindex(y_pred,y_train)
test_scores=resolve(y_pred,sbtrain,y_train,theta=0)
print(test_scores)

0.652027027027027


In [188]:
models=reset()

In [182]:
e2sb["Id"]=X_2["Id"]

In [184]:
m=models['mlp'].set_params(**{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'})
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(sbtrain,y_train)
m.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred=m.predict_proba(e2sb.loc[X_2.index].drop("Id",axis=1))
y_pred=mindex(y_pred,y_2)
test_scores=resolve(y_pred,e2sb.loc[X_2.index],y_2,gt3=E2,theta=0)
print(test_scores)

0.9019607843137255


In [212]:
m=models['mlp'].set_params(**{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'})
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(sbtrain,y_train)
m.fit(XS_train.drop("Id",axis=1),ys_train)

MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50, 50), max_iter=100,
              random_state=0, solver='sgd')

In [213]:
saveObj(m,"ML_EF-resolution.Anaphora")

In [192]:
X_train

,Id,Position,Distance,sameGender,sameNumber,isPerson,isDefinite,isPrepositional,isSubject,isDirectObject,...,precedingPOS_CCONJ,precedingPOS_DET,precedingPOS_NOUN,precedingPOS_PART,precedingPOS_PRON,precedingPOS_PROPN,precedingPOS_PUNCT,precedingPOS_SCONJ,precedingPOS_VERB,precedingPOS_X
0,2007-ertms#0,10.0,8,False,False,False,False,False,True,False,...,0,0,0,0,0,0,0,0,1,0
1,2007-ertms#0,10.0,4,False,True,False,True,False,False,True,...,0,0,0,0,0,0,0,0,1,0
2,2007-ertms#0,10.0,2,False,True,False,False,False,False,False,...,0,0,0,0,0,0,0,0,1,0
3,2007-ertms#1b,31.0,30,True,False,False,False,False,True,False,...,0,0,0,0,0,0,0,0,0,0
4,2007-ertms#1b,31.0,21,True,True,False,False,False,False,True,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,EDRS_MOCReqs#711,29.0,19,True,False,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0
6662,EDRS_MOCReqs#711,29.0,13,True,False,False,True,False,True,False,...,0,0,0,0,0,0,0,0,0,0
6663,EDRS_MOCReqs#711,29.0,4,True,True,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
6664,EDRS_MOCReqs#711,29.0,1,True,False,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0


### Ensemble

#### E1 

In [185]:
def ensembleprobas(y1,y2):
    y=[]
    for p1,p2 in zip(y1,y2):
        y.append(list(np.maximum(p1,p2)))
    return np.array(y)

In [201]:
XS_train

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Id
0,-0.048113,0.043173,-0.063797,0.006073,0.086657,0.084457,0.002498,-0.050632,0.052499,-0.040694,...,0.131097,-0.104955,0.052170,-0.049510,-0.044114,-0.075946,0.009304,-0.339129,-0.062229,HalifaxCombatSystems#36
1,0.049636,0.044323,-0.073470,0.003373,-0.030021,0.044301,0.258999,0.033152,-0.104710,-0.105360,...,0.095254,0.021843,0.120270,-0.016417,0.042146,0.093104,-0.004239,-0.099170,0.023432,MHP#601
2,-0.030563,0.080678,-0.020354,-0.031524,0.048193,-0.056459,0.161643,-0.067842,0.062557,-0.012022,...,0.118148,0.189792,0.207768,-0.046313,-0.089502,-0.008265,0.038456,-0.063006,-0.050217,HalifaxCombatSystems#51
3,-0.171972,0.020777,-0.054643,-0.004264,-0.038089,0.110683,0.074297,0.055485,0.166338,-0.005048,...,0.050840,0.018475,0.000695,-0.098547,-0.212068,0.078286,0.126326,-0.221011,-0.108689,MHP#665
4,-0.031382,0.110554,-0.014402,-0.081880,0.024107,-0.035577,0.138535,-0.074464,-0.001880,0.100875,...,0.058033,-0.019035,0.108372,0.000335,-0.178427,0.041503,0.054729,-0.203780,-0.008174,HalifaxCombatSystems#29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,0.026663,-0.170309,-0.009495,-0.023062,0.158943,-0.114369,0.042996,0.022335,0.130810,0.116944,...,0.148863,-0.040215,-0.012318,-0.102754,-0.154815,0.141562,0.161547,0.027399,-0.107627,0000 - cctns#159c
911,-0.047608,-0.070171,-0.038540,-0.109307,-0.109680,0.126709,0.031687,0.142502,0.270887,-0.017102,...,0.098071,0.068358,0.100297,0.000776,-0.206153,0.038407,0.054824,-0.251801,-0.057734,MHP#649
912,-0.039843,-0.069113,-0.044555,-0.024401,0.098632,-0.034412,0.293480,-0.020448,0.104377,-0.006379,...,0.088783,0.147551,0.058875,-0.111616,-0.079004,0.010577,0.076072,-0.011267,-0.178669,2007-eirene_fun_7-2#61
913,0.009572,0.283225,-0.080852,-0.036663,0.041891,0.037788,0.148302,-0.091053,0.053559,-0.057665,...,0.067896,0.051746,0.170741,0.002853,-0.135354,0.078806,0.090891,-0.077461,-0.026161,HalifaxCombatSystems#2


In [453]:
models=reset()
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
svm=models['svm'].set_params(**{'C': 50, 'gamma': 0.001, 'kernel': 'poly'})
svm.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred1=svm.predict_proba(X_test.drop("Id",axis=1))

XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(sbtrain,y_train)
m=models['mlp'].set_params(**{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'})
m.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred2=m.predict_proba(sbtest.drop("Id",axis=1))


In [464]:
y_pred=ensembleprobas(y_pred1,y_pred2)
y_pred=mindex(y_pred,y_test)
test_scores=resolve(y_pred,X_test,y_test,theta=0)
print(test_scores)

0.8229166666666666


#### E2

In [827]:
models=reset()
XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(X_train,y_train)
svm=models['svm'].set_params(**{'C': 50, 'gamma': 0.001, 'kernel': 'poly'})
svm.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred1=svm.predict_proba(X_2.drop("Id",axis=1))

XS_train,ys_train=RandomUnderSampler(sampling_strategy='not minority', random_state=0).fit_resample(sbtrain,y_train)
m=models['mlp'].set_params(**{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'})
m.fit(XS_train.drop("Id",axis=1),ys_train)
y_pred2=m.predict_proba(e2sb.drop("Id",axis=1))

In [828]:
y_pred=ensembleprobas(y_pred1,y_pred2)
y_pred=mindex(y_pred,y_2)
test_scores=resolve(y_pred,X_2,y_2,theta=0)
print(test_scores)

0.574468085106383


## Baseline

In [538]:
def baselineResolve(xx,context=0):
    ci=[0,0]
    for Id in xx.Id.unique():
        CAs=xx[xx.Id==Id]["Candidate Antecedent"].unique()
        if context==0:
            if len(CAs)>1:
                ci[1]+=1
            elif xx[xx.Id==Id][xx["Candidate Antecedent"]==CAs[0]].Answer.unique()[0]==1:
                ci[0]+=1
            else:
                ci[1]+=1
        elif context==1:
            c=xx[xx.Id==Id].Context.unique()[0]
            sents=sent_tokenize(c.text)
            CIS=[]
            for CA in CAs:
                if CA.text in sents[-1]:
                    CIS.append(CA)
            if len(CIS)!=1:
                ci[1]+=1
            elif xx[xx.Id==Id][xx["Candidate Antecedent"]==CIS[0]].Answer.unique()[0]==1:
                ci[0]+=1
            else:
                ci[1]+=1
    return ci[0]/(ci[0]+ci[1])

### Context

In [540]:
baselineResolve(gt3[gt3.Id.isin(X_test.Id.unique())])

0.00980392156862745

In [531]:
baselineResolve(E2)

0.05172413793103448

### One-Sentence

In [541]:
baselineResolve(gt3[gt3.Id.isin(X_test.Id.unique())],context=1)

0.058823529411764705

In [542]:
baselineResolve(E2)

TypeError: can only concatenate str (not "int") to str

# SpanBERT

In [ ]:
conll=pd.read_csv("CONLL.csv",index_col=0)

In [ ]:
# Load Pre_train Dataset
df = pd.read_csv("Dspans_V21.7.19.csv",sep=";",index_col=0)
df.head()

In [ ]:
E1_Ids=loadObj("E1_Ids.obj")

In [ ]:
for i in df.index:
  if len(ast.literal_eval(df.spans[i]))>1:
    maxspan=()
    mindis=999
    pi=df.context[i].find(df.pronoun[i])
    for s in ast.literal_eval(df.spans[i]):
      d=closest(range(s[0],s[1]+1),pi)
      if d<mindis:
        mindis=d
        maxspan=s
    df.schars[i]=str(list(range(maxspan[0],maxspan[1]+1)))
    df.spans[i]=str([maxspan])

In [ ]:
testdf=df[df.id.isin(E1_Ids)]
traindf=df[~df.id.isin(E1_Ids)]

In [ ]:
fast_tokenizer = BertTokenizerFast.from_pretrained('SpanBERT/spanbert-base-cased')
model = BertForTokenClassification.from_pretrained('SpanBert20CONLLv10.8.21')
tokenizer=fast_tokenizer

In [ ]:
train_data = SpanDetectionData(traindf, fast_tokenizer)#,train=False)
test_data = SpanDetectionData(testdf, fast_tokenizer,train=False)

## SpanBERT NLP

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    save_steps=200,
    logging_steps= 200,          
    save_total_limit = 5,
    #evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=20,              # total number of training epochs
    learning_rate = 2e-5,
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs'
)

nlp_trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    #train_dataset=pretrain_data,         # training dataset
    #eval_dataset=test_data             # evaluation dataset
)



In [ ]:
nlp_predictions=nlp_trainer.predict(test_data)

In [ ]:
ttruncated_predictions,tpredicted_spans=processPred(nlp_predictions,test_data,testdf)

In [ ]:
tests=[]
for t in testdf.schars:
  tests.append(ast.literal_eval(str(t)))

In [ ]:
ddf=df[df.id.isin(loadObj("Dev_Ids.obj"))]
d_data = SpanDetectionData(devdf, fast_tokenizer,train=False)
dpredictions=nlp_trainer.predict(d_data)
truncated_predictions = []
for k in range(len(d_data)):
  inputs_ids = d_data[k]['input_ids']
  truncated_predictions.append(dpredictions.predictions[k][:len(inputs_ids)])
predicted_spans = predict_spans(truncated_predictions, ddf,fast_tokenizer)
ds=[]
for t in ddf.schars:
  devs.append(ast.literal_eval(str(t)))

In [ ]:
X=2
threshold=0.9
tp=0
tn=0
fp=0
fn=0
FPs=[]
for i,p,t,k in zip(ddf.index,truncated_predictions,ds,range(len(d_data))):
    p=list(np.where(softmax(p)[:,1]>0.99)[0])
    c=ddf.context[i]
    nbrS=nbrSpansX(p,x=X)
    if (nbrS==0 or nbrS>1) and not t:
      tp+=1
    elif (nbrS==0 or nbrS>1) and t:
        fp+=1
        ss=findWspans(c,p)
        FPs.append([ddf.id[i],c,ss,findspans(c,t)])
    elif not t and (nbrS==1):
      fn+=1
    else:
    # elif is_slice_in_list(p,t):
      tn+=1
printMetrics(tp,tn,fp,fn)

### Resolution

#### Full match

In [ ]:
ci=[0,0]
for i,p,t in zip(testdf.index, tpredicted_spans,tests):
    if t:
        c=testdf.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        print(preds,true)
        if true in preds:
            ci[0]+=1
        else:
            
            ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

#### Partial match

In [ ]:
ci=[0,0]
for i,p,t in zip(testdf.index, tpredicted_spans,tests):
    if t:
        c=testdf.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            f=False
            for pred in preds:
                if true in pred:
                    ci[0]+=1
                    f=True
                    break
            if not f:
                ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

### E2

In [ ]:
E2df=pd.read_csv("E2-spanBert.csv",sep=";")

In [ ]:
E2df=E2df[~E2df.id.isin(["railway#41","railway#42","railway#43-b","railway#44","railway#45","railway#46-b","railway#47"])]

In [ ]:
E2_data = SpanDetectionData(E2df, fast_tokenizer,train=False)

In [ ]:
e2_nlp_predictions=nlp_trainer.predict(E2_data)

In [ ]:
e2s=[]
for t in E2df.schars:
  e2s.append(ast.literal_eval(str(t)))

In [ ]:
e2truncated_predictions,e2predicted_spans=processPred(e2_nlp_predictions,E2_data,E2df)

In [ ]:
evaluateST(e2truncated_predictions,e2s,X=2,threshold=0.9)

### Resolution

full match

In [ ]:
ci=[0,0]
for i,p,t in zip(E2df.index, e2predicted_spans,e2s):
    if t:
        c=E2df.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

partial match

In [ ]:
ci=[0,0]
for i,p,t in zip(E2df.index, e2predicted_spans,e2s):
    if t:
        c=E2df.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            f=False
            for pred in preds:
                if true in pred:
                    ci[0]+=1
                    f=True
                    break
            if not f:
                ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

## SpanBERT RE

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    save_steps=200,
    logging_steps= 200,          
    save_total_limit = 5,
    #evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=3,              # total number of training epochs
    learning_rate = 2e-5,
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs'
)

retrainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,         # training dataset
    #eval_dataset=test_data             # evaluation dataset
)

retrainer.train()

In [ ]:
testpredictions=retrainer.predict(test_data)
tests=[]
for t in testdf.schars:
  tests.append(ast.literal_eval(str(t)))
test_truncated_predictions = []
for k in range(len(test_data)):
  inputs_ids = test_data[k]['input_ids']
  test_truncated_predictions.append(testpredictions.predictions[k][:len(inputs_ids)])
test_predicted_spans = predict_spans(test_truncated_predictions, testdf,fast_tokenizer)

In [ ]:
X=2
threshold=0.9
tp=0
tn=0
fp=0
fn=0
#FPs=[]
for i,p,t,k in zip(testdf.index,test_truncated_predictions,tests,range(len(test_data))):
    p=list(np.where(softmax(p)[:,1]>threshold)[0])
    nbrS=nbrSpansX(p,x=X)
    if (nbrS==0 or nbrS>1) and not t:
      tp+=1
    elif (nbrS==0 or nbrS>1) and t:
        fp+=1
        #c=testdf.context[i]
#        tokens_info = fast_tokenizer(c, return_offsets_mapping=True)
#        one_idx = np.where(np.argmax(predictions.predictions[k],1)==1)[0]
#        yoyo = [tokens_info['offset_mapping'][j] for j in one_idx if j<len(tokens_info['offset_mapping'])]
#        pred = []
#        for p in yoyo:
#          pred += list(range(p[0],p[1]))
#        FPs.append([testdf.id[i],c,pred,findspans(c,pred),findspans(c,t)])
    elif not t and (nbrS==1):
      fn+=1
    else:
    # elif is_slice_in_list(p,t):
      tn+=1
printMetrics(tp,tn,fp,fn)

### Resoltion

Full match

In [ ]:
ci=[0,0]
for i,p,t in zip(testdf.index, test_predicted_spans,tests):
    if t:
        c=testdf.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

Partial Match

In [ ]:
ci=[0,0]
for i,p,t in zip(testdf.index, test_predicted_spans,tests):
    if t:
        c=testdf.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            f=False
            for pred in preds:
                if true in pred:
                    ci[0]+=1
                    f=True
                    break
            if not f:
                ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

### E2

In [ ]:
e2_re_predictions=retrainer.predict(E2_data)
e2truncated_predictions,e2_predicted_spans=processPred(e2_re_predictions,E2_data,E2df)
e2_predicted_spans_T = predict_spans_T(e2truncated_predictions, E2df,fast_tokenizer,0.9)

In [ ]:
evaluateST(e2truncated_predictions,e2s,X=2,threshold=0.9)

In [ ]:
ci=[0,0]
for i,p,t in zip(E2df.index, e2_predicted_spans,e2s):
    if t:
        c=E2df.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if len(preds)==1:
            if true in preds:
                ci[0]+=1
            else:
                f=False
                for pred in preds:
                    if true in pred:
                        ci[0]+=1
                        f=True
                        break
                if not f:
                    ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))

In [ ]:
ci=[0,0]
for i,p,t in zip(E2df.index, e2_predicted_spans,e2s):
    if t:
        c=E2df.context[i]
        preds=findspans(c,p)
        true=findspans(c,t)[0].strip()
        if true in preds:
            ci[0]+=1
        else:
            ci[1]+=1
print(ci[0]/(ci[0]+ci[1]))